In [35]:
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
from PyPDF2 import PdfFileWriter, PdfFileReader
from requests_html import HTMLSession

In [36]:
# initialize the set of links (unique links)
internal_urls = set()
external_urls = set()
pdf_urls = set()
urls = set()

In [37]:
# This will make sure that a proper scheme (protocol, e.g http or https) and domain name exists in the URL.
def is_valid(url):
    """Checks whether `url` is a valid URL"""
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

In [38]:
def get_all_website_links(url):
    session = HTMLSession()
    response = session.get(url)
    try:
        response.html.render()
    except:
        pass
#     soup = BeautifulSoup(requests.get(url).content, "html.parser")
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(response.html.html, "html.parser")  # this is assumed for javascript loaded pages 
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
#         print(href)
        if href == "" or href is None:
            # href empty tag
            continue
        if href.endswith(".pdf"):
            pdf_urls.add(href)
        
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        if parsed_href.query != "":
            href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path + "?" + parsed_href.query
        else:
            href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        
        
        if not is_valid(href):
            # not a valid URL
            continue
        if href in internal_urls:
            # already in the set
            continue
        if domain_name not in href:
            # its external link
            if href not in external_urls:
                external_urls.add(href)
            continue
        # else its internel links
        urls.add(href)
        internal_urls.add(href)
    return urls
        
        
        
    

In [46]:
total_urls_visited = 0

# help to call give number of times inside the website
def crawl(url, max_urls=10):
    """
    Crawls a web page and extracts all links.
    You'll find all links in `external_urls` and `internal_urls` global set variables.
    params:
        max_urls (int): number of max urls to crawl, default is 10.
    """
    global total_urls_visited
    total_urls_visited += 1
    links = get_all_website_links(url)
    for link in links:
        print(link)
        if total_urls_visited > max_urls:
            break
        crawl(link, max_urls=max_urls)
        print(total_urls_visited)

In [47]:
import argparse
root_url  =  "https://ijser.org/"
if __name__ == "__main__":
   
    crawl(root_url,max_urls = 2)
    
    
    print("[+] Total External links:", len(external_urls))
    print("[+] Total Internal links:", len(internal_urls))
    print("[+] Total:", len(external_urls) + len(internal_urls))
    
#     domain_name = urlparse(root_url).netloc
    
#     # save the internal links to a file
#     with open(f"{domain_name}_internal_links.txt", "w") as f:
#         for internal_link in internal_urls:
#             print(internal_link.strip(), file=f)

#     # save the external links to a file
#     with open(f"{domain_name}_external_links.txt", "w") as f:
#         for external_link in external_urls:
#             print(external_link.strip(), file=f)
        

https://www.ijser.org/ResearchPaperPublishing_February2011.aspx
https://www.ijser.org/ResearchPaperPublishing_February2011.aspx
https://www.ijser.org/ResearchPaperPublishing_February2011.aspx
3
https://www.ijser.org/ResearchPaperPublishing_March2011.aspx
3
https://www.ijser.org/ResearchPaperPublishing_March2011.aspx
[+] Total External links: 6
[+] Total Internal links: 191
[+] Total: 197


In [11]:
parsed = urlparse("https://www.geeksforgeeks.org/k-nearest-neighbours/")
netloc = parsed.netloc
netloc

'www.geeksforgeeks.org'

In [8]:
#Create pdfs and txts 
import os
parsed = urlparse(root_url)
netloc = parsed.netloc

if not os.path.exists("pdfs"):
    os.mkdir("pdfs")

if not os.path.exists("txts"):
    os.mkdir("txts")
    
pdf_dirName = os.path.join("pdfs",netloc.split(".")[1])
txt_dirName = os.path.join("txts",netloc.split(".")[1])

def make_dir(dirName):
    if not os.path.exists(dirName):
        os.mkdir(dirName)
        print("Directory " , dirName ,  " Created ")
    else:
        print("Directory " , dirName ,  " already exists")

make_dir(pdf_dirName)
make_dir(txt_dirName)
make_dir(domain)

Directory  pdfs\org  Created 
Directory  txts\org  Created 


In [26]:
txt_dirName

'txts\\org'

In [48]:
internal_urls = list(internal_urls)
pdf_urls = [iu for iu in internal_urls if iu.endswith(".pdf")]

total_pdfs = len(pdf_urls)
print(f"Total pdfs {total_pdfs}!")

Total pdfs 16!


In [17]:
pdf_urls[0]

'http://www.ijset.net/journal/741.pdf'

In [24]:
def dowload_pdf_save_txt(url):
    print(url)
    pdf_name = url.split("/")[-1]
    print(pdf_name)
    try:
        response = requests.get(url)
        with open(f'{pdf_dirName}/{pdf_name}', 'wb') as f:
            f.write(response.content)
            f.close()
        output = PdfFileWriter()
        inputs = PdfFileReader(open(f'{pdf_dirName}/{pdf_name}', "rb"))
        print (f"{pdf_name} has {inputs.getNumPages()} pages.")
        total_pages = inputs.getNumPages()
        txt = ""
        for i in range(total_pages):
            txt+=inputs.getPage(i).extractText()

        with open(f'{txt_dirName}/{pdf_name.split(".")[0]}.txt', 'w+') as f:
            f.write(txt)
            f.close()
    except:
        pass

In [34]:
import time
t1 = time.time()
count = 0
select_number_of_pdf = 2
for pdf_url in pdf_urls[:select_number_of_pdf]:
    count+=1
    dowload_pdf_save_txt(pdf_url)
print(f"Total {count} pdfs downloaded:")
t2 = time.time()
print(f'!{t2-t1} " seconds take!')
print("Lets see pdfs and txts folder")

http://www.ijset.net/journal/741.pdf
741.pdf
741.pdf has 9 pages.
http://www.ijset.net/journal/2323.pdf
2323.pdf
2323.pdf has 8 pages.
Total 2 pdfs downloaded:
!10.378966093063354 " seconds take!
Lets see pdfs and txts folder


In [87]:
#Convert pdfs to txts
output = PdfFileWriter()
input1 = PdfFileReader(open("ijser.pdf", "rb"))
# print how many pages input1 has:
print ("document1.pdf has %d pages." % input1.getNumPages())
total_pages = input1.getNumPages()

txt = ""
for i in range(total_pages):
    txt+=input1.getPage(i).extractText()

with open(f'{txt_dirName}/one.txt', 'w+') as f:
    f.write(txt)
    f.close()

document1.pdf has 7 pages.


In [ ]:
import argparse
parser = argparse.ArgumentParser(description="Link Extractor Tool with Python")
parser.add_argument("url", help="The URL to extract links from.")
parser.add_argument("-m", "--max-urls", help="Number of max URLs to crawl, default is 5.", default=5, type=int)

args = parser.parse_args()
root_url = args.url
max_urls = args.max_urls



domain_name = urlparse(url).netloc

# save the internal links to a file
with open(f"{domain_name}_internal_links.txt", "w") as f:
    for internal_link in internal_urls:
        print(internal_link.strip(), file=f)

# save the external links to a file
with open(f"{domain_name}_external_links.txt", "w") as f:
    for external_link in external_urls:
        print(external_link.strip(), file=f)